In [4]:
import sqlite3
import pandas as pd

df = pd.read_excel("/Users/feepieper/Library/CloudStorage/OneDrive-Persönlich/Ironhack/Module4/projects/mini-project-dsai-ml-regression/data/regression_data.xls")

conn = sqlite3.connect("house_price_regression.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS house_price_data (
    id INTEGER PRIMARY KEY,
    date TEXT,
    bedrooms INTEGER,
    bathrooms REAL,
    sqft_living INTEGER,
    sqft_lot INTEGER,
    floors REAL,
    waterfront TEXT,
    view INTEGER,
    condition INTEGER,
    grade INTEGER,
    sqft_above INTEGER,
    sqft_basement INTEGER,
    yr_built INTEGER,
    yr_renovated INTEGER,
    zipcode TEXT,
    lat REAL,
    long REAL,
    sqft_living15 INTEGER,
    sqft_lot15 INTEGER,
    price REAL
)
""")

# insert data from excel in table
df.to_sql("house_price_data", conn, if_exists="replace", index=False)

# check if data was inserted
result = pd.read_sql_query("SELECT * FROM house_price_data LIMIT 5", conn)
print(result)


           id                 date  bedrooms  bathrooms  sqft_living  \
0  7129300520  2014-10-13 00:00:00         3       1.00         1180   
1  6414100192  2014-12-09 00:00:00         3       2.25         2570   
2  5631500400  2015-02-25 00:00:00         2       1.00          770   
3  2487200875  2014-12-09 00:00:00         4       3.00         1960   
4  1954400510  2015-02-18 00:00:00         3       2.00         1680   

   sqft_lot  floors  waterfront  view  condition  ...  sqft_above  \
0      5650     1.0           0     0          3  ...        1180   
1      7242     2.0           0     0          3  ...        2170   
2     10000     1.0           0     0          3  ...         770   
3      5000     1.0           0     0          5  ...        1050   
4      8080     1.0           0     0          3  ...        1680   

   sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
0              0      1955             0    98178  47.5112 -122.257   
1         

In [ ]:
# ALTER TABLE: Drop the "date" column --> creatinf table without "date" column
cursor.execute("""
CREATE TABLE IF NOT EXISTS house_price_data_new AS
SELECT id, bedrooms, bathrooms, sqft_living, sqft_lot, floors, waterfront, view, condition,
       grade, sqft_above, sqft_basement, yr_built, yr_renovated, zipcode, lat, long, sqft_living15,
       sqft_lot15, price
FROM house_price_data;
""")

# Drop the old table
cursor.execute("DROP TABLE house_price_data;")

# Rename the new table
cursor.execute("ALTER TABLE house_price_data_new RENAME TO house_price_data;")

conn.commit()

# Check if the column was removed (first 10 rows)
result = pd.read_sql_query("SELECT * FROM house_price_data LIMIT 10;", conn)
print(result)

# Count the total number of rows in the table
row_count = pd.read_sql_query("SELECT COUNT(*) AS total_rows FROM house_price_data;", conn)
print(row_count)

           id  bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  \
0  7129300520         3       1.00         1180      5650     1.0           0   
1  6414100192         3       2.25         2570      7242     2.0           0   
2  5631500400         2       1.00          770     10000     1.0           0   
3  2487200875         4       3.00         1960      5000     1.0           0   
4  1954400510         3       2.00         1680      8080     1.0           0   
5  7237550310         4       4.50         5420    101930     1.0           0   
6  1321400060         3       2.25         1715      6819     2.0           0   
7  2008000270         3       1.50         1060      9711     1.0           0   
8  2414600126         3       1.00         1780      7470     1.0           0   
9  3793500160         3       2.50         1890      6560     2.0           0   

   view  condition  grade  sqft_above  sqft_basement  yr_built  yr_renovated  \
0     0          3      7   

# Amount of rows in dataset

In [13]:
query_row_count = "SELECT COUNT(*) AS total_rows FROM house_price_data;"
row_count = pd.read_sql_query(query_row_count, conn)
print(row_count)

   total_rows
0       21597


# Unique values

In [ ]:
query_bedrooms = "SELECT DISTINCT bedrooms FROM house_price_data ORDER BY bedrooms;"
bedrooms_unique = pd.read_sql(query_bedrooms, conn)
print("Unique bedrooms:")
print(bedrooms_unique)

query_bathrooms = "SELECT DISTINCT bathrooms FROM house_price_data ORDER BY bathrooms;"
bathrooms_unique = pd.read_sql(query_bathrooms, conn)
print("\nUnique bathrooms:")
print(bathrooms_unique)

query_floors = "SELECT DISTINCT floors FROM house_price_data ORDER BY floors;"
floors_unique = pd.read_sql(query_floors, conn)
print("\nUnique floors:")
print(floors_unique)

query_condition = "SELECT DISTINCT condition FROM house_price_data ORDER BY condition;"
condition_unique = pd.read_sql(query_condition, conn)
print("\nUnique condition:")
print(condition_unique)

query_grade = "SELECT DISTINCT grade FROM house_price_data ORDER BY grade;"
grade_unique = pd.read_sql(query_grade, conn)
print("\nUnique grade:")
print(grade_unique)

Unique bedrooms:
    bedrooms
0          1
1          2
2          3
3          4
4          5
5          6
6          7
7          8
8          9
9         10
10        11
11        33

Unique bathrooms:
    bathrooms
0        0.50
1        0.75
2        1.00
3        1.25
4        1.50
5        1.75
6        2.00
7        2.25
8        2.50
9        2.75
10       3.00
11       3.25
12       3.50
13       3.75
14       4.00
15       4.25
16       4.50
17       4.75
18       5.00
19       5.25
20       5.50
21       5.75
22       6.00
23       6.25
24       6.50
25       6.75
26       7.50
27       7.75
28       8.00

Unique floors:
   floors
0     1.0
1     1.5
2     2.0
3     2.5
4     3.0
5     3.5

Unique condition:
   condition
0          1
1          2
2          3
3          4
4          5

Unique grade:
    grade
0       3
1       4
2       5
3       6
4       7
5       8
6       9
7      10
8      11
9      12
10     13


# Top 10 most expensive houses

In [ ]:
query_top10 = """
SELECT id, price
FROM house_price_data
ORDER BY price DESC
LIMIT 10;
"""

top10_houses = pd.read_sql(query_top10, conn)
print(top10_houses)

           id    price
0  6762700020  7700000
1  9808700762  7060000
2  9208900037  6890000
3  2470100110  5570000
4  8907500070  5350000
5  7558700030  5300000
6  1247600105  5110000
7  1924059029  4670000
8  7738500731  4500000
9  3835500195  4490000


# Average price of all houses sold

In [17]:
query_average_price = """
SELECT ROUND(AVG(price),2) AS average_price
FROM house_price_data;
"""

average_price = pd.read_sql(query_average_price, conn)
print(average_price)

   average_price
0      540296.57


# Average price and sqft_living grouped by bedrooms

In [20]:
query_average_price_by_bedrooms = """
SELECT ROUND(AVG(price),2) AS average_price
FROM house_price_data
GROUP BY bedrooms;
"""

average_price_by_bedrooms = pd.read_sql(query_average_price_by_bedrooms, conn)
print(average_price)

query_average_sqft_by_bedrooms = """
SELECT ROUND(AVG(sqft_living),2) AS average_sqft_living
FROM house_price_data
GROUP BY bedrooms;
"""

average_sqft_by_bedrooms = pd.read_sql(query_average_sqft_by_bedrooms, conn)
print(average_sqft_by_bedrooms)

    average_price
0       318239.46
1       401387.75
2       466276.59
3       635564.68
4       786874.13
5       825853.50
6       951447.82
7      1105076.92
8       893999.83
9       820000.00
10      520000.00
11      640000.00
    average_sqft_living
0                888.15
1               1239.75
2               1805.84
3               2554.65
4               3047.55
5               3284.34
6               3974.21
7               3800.00
8               3775.00
9               3706.67
10              3000.00
11              1620.00


# Average Price on waterfront/ no waterfront

In [21]:
query_avg_waterfront = """
SELECT 
    waterfront,
    ROUND(AVG(price), 2) AS avg_price
FROM 
    house_price_data
GROUP BY 
    waterfront;
"""

avg_waterfront_df = pd.read_sql(query_avg_waterfront, conn)
print(avg_waterfront_df)

   waterfront   avg_price
0           0   531762.32
1           1  1662524.18


In [22]:
query_condition_grade = """
SELECT 
    condition,
    ROUND(AVG(grade), 2) AS avg_grade
FROM 
    house_price_data
GROUP BY 
    condition
ORDER BY 
    condition;
"""
condition_grade_df = pd.read_sql(query_condition_grade, conn)
print(condition_grade_df)


   condition  avg_grade
0          1       5.97
1          2       6.54
2          3       7.83
3          4       7.38
4          5       7.32


# Customer Request

In [23]:
query_customer_options = """
SELECT *
FROM house_price_data
WHERE 
    bedrooms IN (3, 4) AND
    bathrooms > 3 AND
    floors = 1 AND
    waterfront = 0 AND
    condition >= 3 AND
    grade >= 5 AND
    price < 300000;
"""

customer_options_df = pd.read_sql(query_customer_options, conn)
print(customer_options_df)


Empty DataFrame
Columns: [id, bedrooms, bathrooms, sqft_living, sqft_lot, floors, waterfront, view, condition, grade, sqft_above, sqft_basement, yr_built, yr_renovated, zipcode, lat, long, sqft_living15, sqft_lot15, price]
Index: []


# List of properties twice of average

In [ ]:
query_expensive_properties = """
SELECT id, price
FROM house_price_data
WHERE price >= 2 * (SELECT AVG(price) FROM house_price_data)
ORDER BY price DESC;
"""

expensive_properties_df = pd.read_sql(query_expensive_properties, conn)
print(expensive_properties_df)


              id    price
0     6762700020  7700000
1     9808700762  7060000
2     9208900037  6890000
3     2470100110  5570000
4     8907500070  5350000
...          ...      ...
1241  1326059182  1090000
1242  7796000095  1090000
1243  4147200040  1090000
1244  1121000357  1090000
1245  8672200110  1090000

[1246 rows x 2 columns]


# View of list of expensive properties

In [30]:
query_create_view = """
CREATE VIEW IF NOT EXISTS expensive_properties_view AS
SELECT id, price
FROM house_price_data
WHERE price >= 2 * (SELECT AVG(price) FROM house_price_data)
ORDER BY price DESC;
"""

expensive_properties_view_df = pd.read_sql("SELECT * FROM expensive_properties_view", conn)
print(expensive_properties_view_df)

              id    price
0     6762700020  7700000
1     9808700762  7060000
2     9208900037  6890000
3     2470100110  5570000
4     8907500070  5350000
...          ...      ...
1241  1326059182  1090000
1242  7796000095  1090000
1243  4147200040  1090000
1244  1121000357  1090000
1245  8672200110  1090000

[1246 rows x 2 columns]


# Difference of Average Price of 3 and 4 bedrooms

In [31]:
query_avg_price_bedrooms = """
SELECT bedrooms, ROUND(AVG(price), 2) AS avg_price
FROM house_price_data
WHERE bedrooms IN (3, 4)
GROUP BY bedrooms
ORDER BY bedrooms;
"""

avg_price_bedrooms_df = pd.read_sql(query_avg_price_bedrooms, conn)
print(avg_price_bedrooms_df)

   bedrooms  avg_price
0         3  466276.59
1         4  635564.68


# Overview of differen locations

In [33]:
query_distinct_zip = """
SELECT DISTINCT(zipcode)
FROM house_price_data;
"""

distinct_zip = pd.read_sql(query_distinct_zip, conn)
print(distinct_zip)

    zipcode
0     98178
1     98125
2     98028
3     98136
4     98074
..      ...
65    98072
66    98188
67    98014
68    98055
69    98039

[70 rows x 1 columns]


# Properties renovated

In [34]:
query_properties_renovated = """
SELECT id
FROM house_price_data
WHERE yr_renovated >0;
"""

properties_renovated  = pd.read_sql(query_properties_renovated , conn)
print(properties_renovated )

             id
0    6414100192
1    9547205180
2    1483300570
3    2450000295
4    3626039325
..          ...
909  7351200295
910   126039256
911  4305600360
912  3319500628
913  1278000210

[914 rows x 1 columns]


# 11th most expensice property

In [35]:
query_rank11_expensive_property = """
SELECT id, price
FROM house_price_data
ORDER BY price DESC
LIMIT 1 OFFSET 10
;
"""

rank11_expensive_property  = pd.read_sql(query_rank11_expensive_property, conn)
print(rank11_expensive_property)

           id    price
0  6065300370  4210000
